In [ ]:
# From Herman's Paper we use the gegravity package
import pandas as pd
import gme as gme
import gegravity as ge

In [ ]:
gravity_data_location = "sample_gravity_data.csv"
grav_data = pd . read_csv ( gravity_data_location )

In [ ]:
# Define GME Estimation Data
gme_data = gme . EstimationData ( grav_data ,
		   imp_var_name ="importer",
	 	   exp_var_name ="exporter",
		   year_var_name = "year",
		   trade_var_name ="trade")

In [ ]:
# Create Gravity Model
gme_model = gme . EstimationModel ( gme_data ,
					lhs_var ="trade",
					rhs_var =["pta", "contiguity", "common_language",
						"lndist", "international"],
					fixed_effects =[["exporter"], ["importer"]])

# Estimate gravity model with PPML
gme_model . estimate ()
# Print results table
gme_model . results_dict ["all"]. summary ()

In [ ]:
# To conduct a counterfactual experiment
ge_model = ge . OneSectorGE ( gme_model , year = "2006",
			              expend_var_name = "E",
			              output_var_name = "Y",
				reference_importer = "DEU",
				sigma = 5)
ge_model . build_baseline ( omr_rescale =10 )
print ( ge_model . baseline_mr )

In [ ]:
# With the baseline solved, the model can be used to conduct counterfactual policy experiments

# Get a copy of the data
exp_data = ge_model . baseline_data . copy ()

# Modify the desired cost measures
exp_data . loc [( exp_data ["importer"]=="CAN") &
		( exp_data ["exporter"]=="JPN") , "pta"] = 1
exp_data . loc [( exp_data ["importer"]=="JPN") & ( exp_data ["importer"]=="JPN") & ( exp_data ["exporter"]=="CAN") , "pta"] = 1
# Supply the counterfactual data to the model
ge_model . define_experiment ( exp_data )
ge_model . simulate ()

In [ ]:
country_results = ge_model . country_results
bilateral_results = ge_model . bilateral_trade_results
ge_model . export_results (name ="CAN_JPN_PTA_experiment")